In [1]:
from pathlib import Path
import json
import os
from pprint import pprint
from rdflib import RDF, BNode, Graph, Literal, URIRef, Namespace
from scipy.optimize import linear_sum_assignment

from buildingmotif.namespaces import BRICK, BACNET
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
from buildingmotif.template_finder.template_finder import calculate_best_bindings_for_template_and_tokens
from buildingmotif.template_finder.classes import Token


PROJECT_DIR = Path(os.getcwd()).resolve().parent

In [2]:
bm = BuildingMOTIF("sqlite://")
BLDG = Namespace("urn:building/")
Library.load(ontology_graph="https://github.com/BrickSchema/Brick/releases/download/nightly/Brick.ttl")
ground_truth = Library.load(ontology_graph=str(PROJECT_DIR / "gabes_points" / "sprucehall.ttl"))
equiment_templates = Library.load(directory=str(PROJECT_DIR / "gabes_points" / "equiment_templates"))
templates = equiment_templates.get_templates()

templates

2023-09-28 12:16:12,356 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7011
2023-09-28 12:16:12,357 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7013
2023-09-28 12:16:12,358 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7015
2023-09-28 12:16:12,359 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7017
2023-09-28 12:16:12,360 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7019
2023-09-28 12:16:12,361 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7045
2023-09-28 12:16:12,362 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7047
2023-09-28 12:16:12,363 | root |  WARNING: Warning: could not find dependee nfdc066aad3d8493f99d488e7940cf5afb7049
2023-09-28 12:16:12,364 | root |  WARNING: Warning: could not find dependee nfdc

[Template(_id=1347, _name='SaTmpEntity', body=<Graph identifier=8d48da19-8148-417c-928e-46827861a549 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x140428ca0>),
 Template(_id=1348, _name='fcu', body=<Graph identifier=593527ab-1386-41c2-8387-4567b1d0ed19 (<class 'rdflib.graph.Graph'>)>, optional_args=['occ_cmd', 'occ_status', 'occ_clg_sp', 'unocc_clg_sp', 'occ_htg_sp', 'unocc_htg_sp'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x140428ca0>),
 Template(_id=1349, _name='room', body=<Graph identifier=fab044b6-54ce-4e04-bc7a-ef72e7818461 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x140428ca0>),
 Template(_id=1350, _name='heating_coil', body=<Graph identifier=0069ccb9-2f90-47c7-8b60-f452990ee926 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMO

In [3]:
with open(PROJECT_DIR / "gabes_points" / "tokens.json") as f:
    labels_and_tokens = json.load(f)

pprint(labels_and_tokens[0])

{'label': ':BuildingName_02:FCU503_ChwVlvPos',
 'tokens': [{'identifier': 'BuildingName',
             'type': 'https://brickschema.org/schema/Brick#Entity'},
            {'identifier': '02',
             'type': 'https://brickschema.org/schema/Brick#Air_Handling_Unit'},
            {'identifier': '503',
             'type': 'https://brickschema.org/schema/Brick#Fan_Coil_Unit'},
            {'identifier': ':BuildingName_02:FCU503_ChwVlvPos',
             'type': 'https://brickschema.org/schema/Brick#Chilled_Water_Valve'}]}


In [4]:
def scalar_cost(cost_dict):
    return cost['edge_cost'] + cost['params_dropped'] + cost['tokens_dropped']

In [5]:
example = [URIRef(x["type"]) for x in labels_and_tokens[0]["tokens"]]
best_templates = {}
mappings = {}

for label_and_tokens in labels_and_tokens[:10]:
    label, tokens = label_and_tokens["label"], label_and_tokens["tokens"]
    print(f"\n=== {label} ===")
    tokens = [Token(classname=URIRef(t['type']), identifier=t['identifier']) for t in tokens]
    print(f"{tokens=}")

    best_template, best_mapping, lowest_cost = calculate_best_bindings_for_template_and_tokens(templates, tokens, verbose=True)
    best_templates[label] = best_template.in_memory_copy()
    mappings[label] = best_mapping.copy()

    print(best_mapping)


=== :BuildingName_02:FCU503_ChwVlvPos ===
tokens=[BuildingName a Entity, 02 a Air_Handling_Unit, 503 a Fan_Coil_Unit, :BuildingName_02:FCU503_ChwVlvPos a Chilled_Water_Valve]
Template costs:
- SaTmpEntity 4.0
- fcu 18.0
- room inf
- heating_coil inf
- cooling_coil 5.0
{name a Fan_Coil_Unit: 503 a Fan_Coil_Unit}

=== :BuildingName_01:FCU336_OccHtgSptFnl ===
tokens=[BuildingName a Entity, 01 a Air_Handling_Unit, 336 a Fan_Coil_Unit, OccHtgSptFnl a Entity]
Template costs:
- SaTmpEntity 4.0
- fcu 20.0
- room inf
- heating_coil inf
- cooling_coil inf
{name a Fan_Coil_Unit: 336 a Fan_Coil_Unit}

=== :BuildingName_02:FCU510_EffOcc ===
tokens=[BuildingName a Entity, 02 a Air_Handling_Unit, 510 a Fan_Coil_Unit, :BuildingName_02:FCU510_EffOcc a Occupancy_Status]
Template costs:
- SaTmpEntity 4.0
- fcu 18.0
- room inf
- heating_coil inf
- cooling_coil inf
{name a Fan_Coil_Unit: 510 a Fan_Coil_Unit}

=== :BuildingName_02:FCU507_UnoccHtgSpt ===
tokens=[BuildingName a Entity, 02 a Air_Handling_Unit

In [6]:
for label, template in best_templates.items():
    print(label)
    print(template.name)
    bindings = {p.name: BLDG[t.identifier] for p, t in mappings[label].items()}
    print(bindings)
    _, g = template.evaluate(bindings).fill(BLDG)
    print(g.serialize())

:BuildingName_02:FCU503_ChwVlvPos
SaTmpEntity
{'name': rdflib.term.URIRef('urn:building/503')}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/503> a brick:Fan_Coil_Unit ;
    brick:hasPoint <urn:building/sat_0f807d58> .

<urn:building/sat_0f807d58> a brick:Supply_Air_Temperature_Sensor .


:BuildingName_01:FCU336_OccHtgSptFnl
SaTmpEntity
{'name': rdflib.term.URIRef('urn:building/336')}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/336> a brick:Fan_Coil_Unit ;
    brick:hasPoint <urn:building/sat_3a91cc4d> .

<urn:building/sat_3a91cc4d> a brick:Supply_Air_Temperature_Sensor .


:BuildingName_02:FCU510_EffOcc
SaTmpEntity
{'name': rdflib.term.URIRef('urn:building/510')}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/510> a brick:Fan_Coil_Unit ;
    brick:hasPoint <urn:building/sat_653e99c3> .

<urn:building/sat_653e99c3> a brick:Supply_Air_Temperature_Sensor .


:BuildingName_02:FCU507_UnoccHtgSpt
SaTmpEntity
{'na

/Users/heslinge/Repos/BuildingMotif/buildingmotif/dataclasses/template.py:372: UserWarning: Parameters "sat" were not provided during evaluation
  warnings.warn(
